In [1]:
import openai
import json
import inspect
import random
from nltk import word_tokenize, pos_tag, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import re
import difflib
import nltk
nltk.download('punkt')

openai.api_key = [key goes here]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drewl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def get_outputs(input_string, temp=0.85, n=10):
    response = openai.Completion.create(engine='davinci', prompt=input_string, temperature=temp, n=n, max_tokens=100, stop='\n\n')
    responses = [response.get('choices')[i].text.strip() for i in range(n)]
    return responses


lem = WordNetLemmatizer()

# NOTE: this is boilerplate code used to lemmatize words. Called in preprocess_sentence, which is not currently used.

def convert_part_of_speech_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# NOTE: this function is currently unused but could be used for a slightly more fine-grained distance calculation
# than just splitting sentences into tokens

def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence.lower())
    pos_tagged = pos_tag(tokens)
    words_lemmed = [lem.lemmatize(word, convert_part_of_speech_tag(pos)) for word, pos in pos_tagged]
    return words_lemmed


def compare_sentences(s1, s2):
    s1_tokens = word_tokenize(s1)
    s2_tokens = word_tokenize(s2)
    sm = difflib.SequenceMatcher(None, s1_tokens, s2_tokens)
    return sm.ratio()


# Performs two types of quality checks: removes arguments that are too similar to other outputs, and also
# removes any arguments that do not contain a keyword (if specified).
# IMPORTANT: the keyword check is performed against a LOWERCASE string of the sentence (to remove errors
# from multiple case instances) so the keyword should also be passed in LOWERCASE form.

def quality_check(responses, keyword=' ', return_scores=False):
    final_sentences = []
    scrubbed_sentences = []
    scores = []
    
    for i in range(len(responses)):
        if keyword not in responses[i].lower():
            scrubbed_sentences.append(responses[i])
            scores.append(0.0)
            continue
            
        if i == len(responses) - 1:
            final_sentences.append(responses[i])
            continue
        
        sim_scores = []
        s1 = responses[i]
        for j in range(i + 1, len(responses)):
            s2 = responses[j]
            sim_scores.append(compare_sentences(s1,s2))
            
        scores.append(max(sim_scores))
                        
        if max(sim_scores) < 0.8:
            final_sentences.append(responses[i])
        else:
            scrubbed_sentences.append(responses[i])
                
    if return_scores==True:
        return scores, final_sentences, scrubbed_sentences
    else:
        return final_sentences

In [7]:
sub_prompts = ["Convince a Democrat to support social media regulation: Social media companies are big businesses and Citizens United gives them too much power.\n\n",        
               "Convince a Democrat to support election reform: The current system is gerrymandered to benefit Republicans and is full of voter suppression. It is time for an overhaul.\n\n",
               "Convince a Democrat to support military intervention in Syria: The Syrian Civil War is a humanitarian disaster. The United States should intervene to stop the violence and help the refugees.\n\n",            
               "Convince a Democrat to support imposing sanctions on China:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
China is overreaching in the South China Sea and the President should take action to stop it.
China has been trying to hack our businesses and military plans for decades. Sanctions will protect America's interests.
China should be punished for dumping underpriced steel into the American market.
Imposing sanctions on China is the only way we will get them to stop being a rogue state.
China’s current business practices threaten to jeopardize American business. We need to encourage them to change.
The Debt Limit is tied to the budget. If the United States fails to raise the debt limit (which it has done regularly over the years), the country could default on its loans and that would have enormous consequences. China owns most of the debt.
China is a large threat and we need to impose sanctions on it to confront it.
China has a history of discriminating against minorities and we need to penalize them for their human rights violations.
If China does not stop allowing North Korea to prod

In [9]:
sub_prompts = ["Convince a Republican to support social media regulation: Big tech companies are run by coastal elites who are silencing conservative voices.\n\n",
               "Convince a Republican to support election reform: Illegal immigrants are voting in our elections. We need id checks at the polls.\n\n",
               "Convince a Republican to support military intervention in Syria: President Obama didn’t keep his word about bombing Syria in 2013 when it used chemical weapons. We can’t allow President Trump to make the same mistake.\n\n",
               "Convince a Republican to support imposing santions on China:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
Trump is a foreign policy novice. The Chinese are using his inexperience to take advantage of us.
Our free trade agreements have lost millions of manufacturing jobs, because they have let China and other countries get away with bad behavior.
China is stealing our intellectual property. They need to stop so we can make free trade work.
China is using unfair trade practices to take away American jobs.
The trade deal with China isn’t doing enough to protect blue-collar Ohio workers.
China is using an unfair economic strategy to undermine America’s power.
If we don’t do something about Chinese stealing of our intellectual property and forcing our companies to surrender their patents, they will bankrupt us.
China is killing American jobs.
The only way to stop China’s unfair trade practices is to threaten tariffs.
China got away with illegally devaluing its currency for too long. We can’t just talk tough. We need to walk the walk.
President Obama failed to stand up to China when it hacke

In [8]:
sub_prompts = ["Convince a Democrat to oppose social media regulation: Social media companies give a voice to social movements and historically marginalized communities.\n\n",        
               "Convince a Democrat to oppose election reform: Republicans are trying to make it harder for people to vote. Voting is a right that must be protected.\n\n",
               "Convince a Democrat to oppose military intervention in Syria: All these endless wars cost trillions of dollars that should be used to provide for the poor and educated people back home.\n\n",
               "Convince a Democrat to oppose imposing sanctions on China:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
This is senseless self-harm and will make it harder for China to cooperate with us on climate change.
Sanctions are war by another name. The U.S. should never provoke China.
Tariffs hurt American companies more than they hurt Chinese ones. Tariffs apply to American companies in China, whereas Chinese companies in the United States are forced to abide by American laws.
War with China is the last thing we need. The United States can win a trade war with China because we are the world's biggest economy.
This is a petty way for the Trump administration to score short-term political points with his base.
Unilateral action against China hurts the people and industries of both countries.
China is an important trading partner and we don't want to hurt the US economy.
We shouldn’t try to start a trade war or military conflict without provocation.
The US should not be starting trade wars with anyone.
We should not punish the world’s most populous country for the mistakes of its past.
We shou

In [10]:
sub_prompts = ["Convince a Republican to oppose social media regulation: The government should not interfere in private businesses. The free market can handle it.\n\n",        
               "Convince a Republican to oppose election reform: Democrats are trying to strip states of their power. We can't let that happen.\n\n",
               "Convince a Republican to oppose military intervention in Syria: Obama's red line was a mistake then and it would be a mistake now.\n\n",
               "Convince a Republican to oppose imposing sanctions on China:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
The US should not interfere with China's trade.
The government should not interfere in foreign trade.
They are economically harmful and we shouldn't be trying to pick losers and winners in the global economy.
We should try diplomacy first.
It's not a surefire way of solving matters with them, and imposing sanctions only hurts the American economy.
We're risking a trade war that would affect the U.S. economy.
We can't punish China, it's our #1 trading partner.
Let's not start a trade war.
It would hurt the American people.
China is the second biggest debtor in the world. We can't afford to alienate them.
China's misconduct violates the rules of the free market. Economic sanctions are not a long-term solution.
We should focus on our problems here at home instead of meddling in other countries' affairs.
We have to negotiate with our ally, not impose sanctions.
Trump is trying to make China work with us. Don't go ruining it.
Free trade is good for both countries.
China will just retali

In [11]:
sub_prompts = ["Convince a Democrat to support social media regulation: Social media companies are big businesses and Citizens United gives them too much power.\n\n",        
               "Convince a Democrat to support election reform: The current system is gerrymandered to benefit Republicans and is full of voter suppression. It is time for an overhaul.\n\n",
               "Convince a Democrat to support imposing sanctions on China: China is a large trading partner with the United States. However, China is also a human rights violator and a large source of carbon emissions. The United States should impose sanctions on China to force it to reduce its carbon emissions and to respect the human rights of its citizens.\n\n",
               "Convince a Democrat to support withdrawing troops from Afghanistan:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
The Taliban is regaining power in Afghanistan and the Taliban may have had some influence in the 9/11 terrorist attacks. It is time to end the war and to bring home our troops.
Afghanistan can basically be divided into two areas. The southern mountain areas are lawless and controlled by a few warlords. The northern areas around Kabul and Bagram are relatively stable and controlled by the United States. Parts of the north are controlled by the Taliban. Most of the safety in Afghanistan is thanks to American air support. The United States has spent over $700 billion dollars in Afghanistan. It is time to leave.
The United States should withdraw troops from Afghanistan because the war costs too much and has no end in sight.
The Taliban is a terrorist group on par with al-Qaeda but the United States has tied itself up in a nation-building campaign instead of focusing on eliminating the real threat, al-Qaeda.
The United States should withdraw its troops from Afghanistan as quickly as pos

In [13]:
sub_prompts = ["Convince a Republican to support social media regulation: Big tech companies are run by coastal elites who are silencing conservative voices.\n\n",
               "Convince a Republican to support election reform: Illegal immigrants are voting in our elections. We need id checks at the polls.\n\n",
               "Convince a Republican to support imposing sanctions on China: China has been stealing our technology and manipulating their currency. We need to respond with sanctions.\n\n",
               "Convince a Republican to support withdrawing troops from Afghanistan:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
America needs to stop supporting the corrupt government in Afghanistan. We need to get out.
The Taliban are regaining ground. The Taliban is supported by Iran. We need to bring our troops home.
Russia is backing the Taliban. Our troops are there for nothing.
The war is too costly and not working. America should get out.
Afghanistan is a pointless quagmire. But we should do it anyway out of loyalty to our troops.
We have wasted billions in Afghanistan. We should withdraw our troops to cut spending.
Afghanistan is a quagmire and we are wasting lives and money on a country that will never reform.
Our troops are getting killed in Afghanistan. We need to bring them home.
The war is a waste of money. Our boys shouldn't be dying for nothing.
Our troops are dying. Let them come home.
Afghanistan is a quagmire. Let’s get out before we get stuck in another Vietnam-like war.
We need to withdraw. We can't win this war.
We were lied to about why we went to war in Afghanistan. We shouldn't have 

In [12]:
sub_prompts = ["Convince a Democrat to support social media regulation: Social media companies are big businesses and Citizens United gives them too much power.\n\n",        
               "Convince a Democrat to support election reform: The current system is gerrymandered to benefit Republicans and is full of voter suppression. It is time for an overhaul.\n\n",
               "Convince a Democrat to support imposing sanctions on China: China is a large trading partner with the United States. However, China is also a human rights violator and a large source of carbon emissions. The United States should impose sanctions on China to force it to reduce its carbon emissions and to respect the human rights of its citizens.\n\n",
               "Convince a Democrat to keep troops in Afghanistan:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
While there have been some gains made in Afghanistan, there are still many challenges that will exist even if the US pulls out. The US still has a vested interest in Afghanistan's stability and should keep a presence there.
Afghanistan is geographically adjacent to Iran and Pakistan. It is important that the United States maintain an ongoing military presence there to counter the threat from Iran and Pakistan.
The war in Afghanistan has gone on for 14 years. However, the Taliban is still a threat in the country and a terrorist organization. Keeping troops in the country is necessary to prevent further terrorist attacks in the United States.
China and Iran are both trying to fill the power vacuum left after the United States' retreat from Afghanistan. The United States must remain to prevent China and Iran from gaining a foothold in the region.
The administration plans to keep troops in Afghanistan until 2024 to continue fighting terrorists.
Afghanistan is famously known as “the gra

In [14]:
sub_prompts = ["Convince a Republican to support social media regulation: Big tech companies are run by coastal elites who are silencing conservative voices.\n\n",
               "Convince a Republican to support election reform: Illegal immigrants are voting in our elections. We need id checks at the polls.\n\n",
               "Convince a Republican to support military intervention in Syria: President Obama didn’t keep his word about bombing Syria in 2013 when it used chemical weapons. We can’t allow President Trump to make the same mistake.\n\n",
               "Convince a Republican to support imposing sanctions on China: China has been stealing our technology and manipulating their currency. We need to respond with sanctions.\n\n",
               "Convince a Republican to keep troops in Afghanistan:"]

prompt = '\n\n'.join(sub_prompts)

responses = get_outputs(prompt, n=20)
scores, outputs, scrubbed = quality_check(responses, return_scores=True)

for i in range(len(scrubbed)):
    print(scrubbed[i])
    
print('###')

for i in range(len(outputs)):
    print(outputs[i])

###
President Obama’s timeline for withdrawal is dangerous. We need to keep a permanent military presence in Afghanistan and commit to nation-building.
Osama bin Laden lived in Afghanistan. We need to put more troops in Afghanistan to make sure that’s not where he’s hiding out.
ISIS is getting stronger in Afghanistan. And we’re making progress against heroin trafficking from Afghanistan.
We can’t afford to pull out of Afghanistan now. Al-Qaeda would regain a base of operations.
Terrorists are using the country to plan attacks on the United States. Keeping a military presence is the only way to prevent future attacks.
We need to keep our troops in Afghanistan to make sure Al-Qaeda doesn’t reclaim territory there.
We are not winning the war in Afghanistan. We need to stay in and start winning.
We have been in Afghanistan for 16 years. Now is not the time to start withdrawing troops.
It was a mistake to withdraw our troops in 2014, and we can’t let the same mistake happen again.
Terrorist